In [2]:

import pandas as pd
from datetime import datetime, timezone
import sqlite3


In [3]:
# Load raw data from Assignment A (file.py)
try:
    posts_df = pd.read_csv('posts.csv')
    comments_df = pd.read_csv('comments.csv')
    print(f"Posts: {len(posts_df)} rows")
    print(f"Comments: {len(comments_df)} rows")
    print("\nPosts columns:", posts_df.columns.tolist())
    print("Comments columns:", comments_df.columns.tolist())
    
except FileNotFoundError:
    print("Error")

Posts: 50 rows
Comments: 1706 rows

Posts columns: ['PostID', 'Title', 'Author', 'Score', 'NumComments', 'CreatedUTC']
Comments columns: ['CommentID', 'PostID', 'Author', 'Score', 'Body', 'CreatedUTC']


In [4]:
#remove [deleted] or [removed] authors and titles
posts_clean = posts_df[
    (~posts_df['Author'].isin(['[deleted]', '[removed]', 'None'])) &
    (~posts_df['Title'].str.contains(r'\[deleted\]|\[removed\]', case=False, na=False))
].copy()

# remove [deleted], [removed] content and empty bodies
comments_clean = comments_df[
    (~comments_df['Author'].isin(['[deleted]', '[removed]', 'None'])) &
    (~comments_df['Body'].isin(['[deleted]', '[removed]'])) &
    (comments_df['Body'].notna()) &
    (comments_df['Body'].str.strip() != '')
].copy()

print(f"Posts before cleaning: {len(posts_df)}")
print(f"Posts after cleaning: {len(posts_clean)} (removed {len(posts_df) - len(posts_clean)})")
print(f"Comments before cleaning: {len(comments_df)}")
print(f"Comments after cleaning: {len(comments_clean)} (removed {len(comments_df) - len(comments_clean)})")

Posts before cleaning: 50
Posts after cleaning: 50 (removed 0)
Comments before cleaning: 1706
Comments after cleaning: 1689 (removed 17)


In [5]:
def convert_utc_to_datetime(utc_timestamp):
    return datetime.fromtimestamp(utc_timestamp, tz=timezone.utc)

# Convert timestamps for posts
posts_clean['CreatedDate'] = posts_clean['CreatedUTC'].apply(convert_utc_to_datetime)
posts_clean['Date'] = posts_clean['CreatedDate'].dt.date
posts_clean['Hour'] = posts_clean['CreatedDate'].dt.hour

# Convert timestamps for comments
comments_clean['CreatedDate'] = comments_clean['CreatedUTC'].apply(convert_utc_to_datetime)
comments_clean['Date'] = comments_clean['CreatedDate'].dt.date

print("Timestamps converted successful")
print("Sample converted dates:")
print(f"Posts: {posts_clean['CreatedDate'].head(2).tolist()}")
print(f"Comments: {comments_clean['CreatedDate'].head(2).tolist()}")

Timestamps converted successful
Sample converted dates:
Posts: [Timestamp('2025-07-19 05:46:21+0000', tz='UTC'), Timestamp('2025-07-19 05:39:44+0000', tz='UTC')]
Comments: [Timestamp('2025-07-19 05:52:29+0000', tz='UTC'), Timestamp('2025-07-19 05:54:26+0000', tz='UTC')]


In [6]:
# Add comment length fields
comments_clean['CommentLength_chars'] = comments_clean['Body'].str.len()
comments_clean['CommentLength_words'] = comments_clean['Body'].str.split().str.len()

# Add post title length fields
posts_clean['TitleLength_chars'] = posts_clean['Title'].str.len()
posts_clean['TitleLength_words'] = posts_clean['Title'].str.split().str.len()

print("Calculated fields added")
print(f"Average comment length: {comments_clean['CommentLength_words'].mean():.1f} words")
print(f"Average title length: {posts_clean['TitleLength_words'].mean():.1f} words")

# Show sample 
print("\nSample data with new fields:")
print(comments_clean[['CommentID', 'CommentLength_chars', 'CommentLength_words']].head(3))

Calculated fields added
Average comment length: 35.3 words
Average title length: 11.7 words

Sample data with new fields:
  CommentID  CommentLength_chars  CommentLength_words
0   n3y7t98                   78                   16
1   n3y8162                  329                   45
2   n3y7e19                  220                   10


In [8]:
#Export cleaned data 

# Save to CSV files
posts_clean.to_csv('posts_cleaned.csv', index=False)
comments_clean.to_csv('comments_cleaned.csv', index=False)

# Save to SQLite database
conn = sqlite3.connect('reddit_data_cleaned.db')
posts_clean.to_sql('posts', conn, if_exists='replace', index=False)
comments_clean.to_sql('comments', conn, if_exists='replace', index=False)
conn.close()

print("\nFiles created:")
print("\n" + "="*50)
print("ASSIGNMENT B SUMMARY")
print("="*50)
print(f"\nFinal cleaned data: {len(posts_clean)} posts, {len(comments_clean)} comments")


Files created:

ASSIGNMENT B SUMMARY

Final cleaned data: 50 posts, 1689 comments
